In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import scipy.io as sio
import anndata as ad
import seaborn as sns
import os as os
import sys as sys
sys.path.append('/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/')
from scRNA_utils import *
import operator as op
import matplotlib.colors as mcolors
data_dir_NHDP = "/home/qiuaodon/Desktop/project_data_new/"
data_dir = "/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/CD4_CD8_final_version/"

## check the patients I can use with enough cells in subtypes


In [2]:
adata_T = sc.read(data_dir_NHDP + '1863-counts_cells_cohort1_T_cells_annotated.h5ad')

In [3]:
# get adata_CD4EX and adata_CD8EX
adata_CD4EX = adata_T[adata_T.obs['cell_type'] == 'CD4 EX', :]
adata_CD8EX = adata_T[adata_T.obs['cell_type'] == 'CD8 EX', :]

In [4]:
# check the number of cells for sample_id, find the patient_ids with more than 5 cells in both pre and on timepoints
cell_counts = adata_CD4EX.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()

# Filter patients with more than 5 cells in both 'pre' and 'on' timepoints
valid_patients_CD4EX = cell_counts[(cell_counts['pre'] > 5) & (cell_counts['on'] > 5)].index.tolist()
valid_patients_CD4EX = set(valid_patients_CD4EX)
# check the number of cells for sample_id, find the patient_ids with more than 5 cells in both pre and on timepoints in CD8EX
cell_counts = adata_CD8EX.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()

# Filter patients with more than 5 cells in both 'pre' and 'on' timepoints
valid_patients_CD8EX = cell_counts[(cell_counts['pre'] > 5) & (cell_counts['on'] > 5)].index.tolist()
valid_patients_CD8EX = set(valid_patients_CD8EX)

/tmp/ipykernel_3303816/2168074212.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cell_counts = adata_CD4EX.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()
/tmp/ipykernel_3303816/2168074212.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cell_counts = adata_CD8EX.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()


In [7]:
adata_Fibro = sc.read(data_dir_NHDP + '1863-counts_cells_cohort1_Fibro_cells.h5ad')

In [ ]:
# check the number of cells for sample_id, find the patient_ids with more than 5 cells in both pre and on timepoints
cell_counts = adata_Fibro.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()

# Filter patients with more than 5 cells in both 'pre' and 'on' timepoints
valid_patients_Fibro = cell_counts[(cell_counts['pre'] > 10) & (cell_counts['on'] > 10)].index.tolist()
valid_patients_Fibro = set(valid_patients_Fibro)

/tmp/ipykernel_3303816/3106465870.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cell_counts = adata_M1.obs.groupby(['patient_id', 'timepoint'])['sample_id'].count().unstack()


In [ ]:
# filter adata_Fibro using valid_patients_Fibro
# adata_Fibro = adata_Fibro[adata_Fibro.obs['patient_id'].isin(valid_patients_Fibro), :]
adata_Fibro.obs['expansion'].value_counts()

expansion
NE    16507
E      3000
Name: count, dtype: int64

In [ ]:
adata_Fibro.obs['timepoint'].value_counts()

timepoint
on     12691
pre     9759
Name: count, dtype: int64

# get the DEGs of filtered patients of CD4EX and B

In [ ]:
Patients_CD4EX_Fibro = list(valid_patients_CD4EX & valid_patients_Fibro)
adata_CD4EX_1 = adata_CD4EX[adata_CD4EX.obs['patient_id'].isin(Patients_CD4EX_Fibro), :]
adata_Fibro_1 = adata_Fibro[adata_Fibro.obs['patient_id'].isin(Patients_CD4EX_Fibro), :]

In [ ]:
adata_Fibro_1

View of AnnData object with n_obs × n_vars = 7106 × 5000
    obs: 'patient_id', 'sample_id', 'timepoint', 'treatment', 'expansion', 'BC_type', 'cellType', 'n_genes', 'n_counts', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'cell_type'
    var: 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'means', 'dispersions', 'dispersions_norm', 'highly_variable'
    uns: 'cell_type_colors', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'patient_id_colors', 'pca', 'rank_genes_groups', 'timepoint_colors', 'treatment_colors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [16]:
DEG_1 = paird_ttest(adata_CD4EX_1, condition_key = 'timepoint', sample_id_col = 'sample_id', patient_id_col = 'patient_id')
DEG_1 = DEG_1[DEG_1['pval'] < 0.01]

Input adata is not pseudo-bulk RNA data. Convert to pseudo-bulk RNA data.


/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/scRNA_utils.py:490: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  res_df['pval'] = res_df['pval'].replace([np.inf, -np.inf], np.nan)  # Replace infinite values with NaN


In [18]:
# remove the DEGs with . in the gene names
DEG_1 = DEG_1[~DEG_1.index.str.contains('\.')]

In [ ]:
DEG_2 = paird_ttest(adata_Fibro_1, condition_key = 'timepoint', sample_id_col = 'sample_id', patient_id_col = 'patient_id')
DEG_2 = DEG_2[DEG_2['pval'] < 0.01]

Input adata is not pseudo-bulk RNA data. Convert to pseudo-bulk RNA data.


/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/scRNA_utils.py:490: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  res_df['pval'] = res_df['pval'].replace([np.inf, -np.inf], np.nan)  # Replace infinite values with NaN


In [20]:
DEG_2 = DEG_2[~DEG_2.index.str.contains('\.')]

## IVA

In [21]:
adata_CD4EX_1

View of AnnData object with n_obs × n_vars = 5265 × 5000
    obs: 'patient_id', 'sample_id', 'timepoint', 'treatment', 'expansion', 'BC_type', 'cellType', 'n_genes', 'n_counts', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'cell_type'
    var: 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'cell_type_colors', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [ ]:
adata_Fibro_1

View of AnnData object with n_obs × n_vars = 7106 × 5000
    obs: 'patient_id', 'sample_id', 'timepoint', 'treatment', 'expansion', 'BC_type', 'cellType', 'n_genes', 'n_counts', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'cell_type'
    var: 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'means', 'dispersions', 'dispersions_norm', 'highly_variable'
    uns: 'cell_type_colors', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'patient_id_colors', 'pca', 'rank_genes_groups', 'timepoint_colors', 'treatment_colors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [ ]:
adata_1_pseudo = scRNA2PseudoBulkAnnData(adata_CD4EX_1, sample_id_col='sample_id')
adata_2_pseudo = scRNA2PseudoBulkAnnData(adata_Fibro_1, sample_id_col='sample_id')

In [ ]:
gene_1 = DEG_1.index.tolist()
gene_2 = DEG_2.index.tolist()

# Filter out invalid gene names
gene_1 = [gene for gene in gene_1 if gene in adata_1_pseudo.var_names]
gene_2 = [gene for gene in gene_2 if gene in adata_2_pseudo.var_names]

gene_1_matrix = adata_1_pseudo[:, gene_1].X
gene_2_matrix = adata_2_pseudo[:, gene_2].X
gene_1_df = pd.DataFrame(gene_1_matrix, columns=gene_1, index=adata_1_pseudo.obs['sample_id'])
gene_2_df = pd.DataFrame(gene_2_matrix, columns=gene_2, index=adata_2_pseudo.obs['sample_id'])
gene_1_df.columns = [i + '_T' for i in gene_1_df.columns]
gene_2_df.columns = [i + '_Fibro' for i in gene_2_df.columns]
gene_df = gene_1_df
gene_df = pd.merge(gene_df, gene_2_df, on='sample_id')
gene_df['treatment'] = gene_df.index.str.contains('On').astype(int)
# get patient id from sample id remove the _On or _Pre
gene_df['patient_id'] = gene_df.index.str.replace('_On', '').str.replace('_Pre', '')

In [26]:
# export the gene_df to csv
gene_df.to_csv(data_dir + 'gene_df_CD4EX_Fibro.csv')

## load in IV result and perform CIT

In [31]:
DEG_pairs = pd.read_excel(data_dir + 'IV_regression_results_with_FisherZ_CD4EX_Fibro.xlsx')

In [32]:
DEG_pairs = DEG_pairs.rename(columns = {'gene_T':'g1', 'gene_cell':'g2'})
calculate_gene_correlation(DEG_pairs, adata_1_pseudo, adata_2_pseudo)

/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/scRNA_utils.py:914: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.12191662186747022' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  results.loc[index, 'g1vsg2_correlation'] = corr_g1_g2


,g1,g2,p_value_iv,r_squared_iv,p_value_ci,q_value_iv,q_value_ci,g1vsg2_correlation
0,ADGRG1_T,ADAMTS5_Fibro,0.246416,-3.306583,0.004717,0.250689,0.015405,0.121917
1,ADGRG1_T,ANGPTL4_Fibro,0.200355,-2.288291,0.021657,0.209195,0.039245,0.125946
2,ADGRG1_T,APOLD1_Fibro,0.165181,-4.907297,0.000293,0.178377,0.002668,0.164342
3,ADGRG1_T,ARHGEF7_Fibro,0.214458,-3.647463,0.003077,0.221810,0.011799,-0.017977
4,ADGRG1_T,ARMCX4_Fibro,0.133839,-1.876118,0.020900,0.152464,0.038385,-0.381479
...,...,...,...,...,...,...,...,...
8443,TUBA3D_T,WDR6_Fibro,0.118975,-0.742957,0.006282,0.140182,0.018172,0.068940
8444,TUBA3D_T,WSB1_Fibro,0.051299,-0.324546,0.053593,0.094913,0.074515,-0.252870
8445,TUBA3D_T,YOD1_Fibro,0.013442,-0.533774,0.004842,0.069416,0.015600,-0.378634
8446,TUBA3D_T,ZC2HC1C_Fibro,0.070874,-0.672126,0.009809,0.107601,0.023702,0.081165


In [33]:
# Combine the two conditions using the logical OR operator
DEG_pairs = DEG_pairs[(DEG_pairs['g1vsg2_correlation'] > 0.4) | (DEG_pairs['g1vsg2_correlation'] < -0.4)]
DEG_pairs = DEG_pairs[(DEG_pairs['p_value_ci'] > 0.1)]
# sort the DEG_pairs by g1vsg2_correlation
DEG_pairs = DEG_pairs.sort_values(by='g1vsg2_correlation', ascending=False)
DEG_pairs

,g1,g2,p_value_iv,r_squared_iv,p_value_ci,q_value_iv,q_value_ci,g1vsg2_correlation
3461,ID3_T,FAM157C_Fibro,0.008785,0.614249,0.565014,0.065328,0.575297,0.814237
907,CD38_T,ERRFI1_Fibro,0.000211,0.502224,0.192537,0.026975,0.213599,0.762372
2232,F2R_T,GBP1_Fibro,0.024931,0.532527,0.844399,0.079742,0.848719,0.742757
1358,CRTAM_T,IL18BP_Fibro,0.007925,0.508993,0.758154,0.065108,0.764124,0.728846
4905,MAFB_T,RGS1_Fibro,0.002309,0.319683,0.108664,0.055915,0.130378,0.724028
...,...,...,...,...,...,...,...,...
2134,ERN1_T,DDIT4L_Fibro,0.002159,0.308629,0.280279,0.055915,0.298625,-0.609782
8212,ZFP36L1_T,F3_Fibro,0.000395,0.379393,0.480622,0.035871,0.492575,-0.636944
5120,MTHFD2_T,CRABP2_Fibro,0.015358,0.416059,0.845291,0.071847,0.849515,-0.660662
8258,ZFP36L1_T,TNFAIP3_Fibro,0.000251,0.446267,0.831634,0.029042,0.836543,-0.681651


In [34]:
lrpair = pd.read_csv('/home/qiuaodon/Desktop/project_data_new/lr_network_unique.tsv', sep='\t')

In [35]:
# remove the ppi_prediction and ppi_prediction_go
lrpair = lrpair[lrpair['database'] != 'ppi_prediction']
lrpair = lrpair[lrpair['database'] != 'ppi_prediction_go']
lrpair = lrpair[['from', 'to']]
# change from as L and to as R
lrpair.columns = ['L', 'R']
lrpair

,L,R
0,CXCL1,CXCR2
1,CXCL2,CXCR2
2,CXCL3,CXCR2
3,CXCL5,CXCR2
4,PPBP,CXCR2
...,...,...
1387,COL2A1,SDC1
1388,LAMA1,SV2A
1389,COL2A1,GP6
1390,LAMA1,DAG1


In [36]:
# Check expression in adata_1_pseudo
expressed_genes_1 = adata_1_pseudo.var_names[adata_1_pseudo.X.sum(axis=0) > 0].tolist()

# Check expression in adata_2_pseudo
expressed_genes_2 = adata_2_pseudo.var_names[adata_2_pseudo.X.sum(axis=0) > 0].tolist()

# Filter lrpair where both L and R are expressed in their respective datasets
lrpair = lrpair[
    (lrpair['L'].isin(expressed_genes_1)) & 
    (lrpair['R'].isin(expressed_genes_2))
].reset_index(drop=True)
lrpair

,L,R
0,CXCL12,CXCR4
1,CCL17,CCR4
2,CCL22,CCR4
3,CCL19,CCR7
4,CCL21,CCR7
...,...,...
160,COL2A1,ITGA1
161,COL2A1,ITGA3
162,COL2A1,ITGA11
163,COL2A1,ITGB8


## CIT Fisherz

In [37]:
results = CIT_test(DEG_pairs, lrpair, adata_1_pseudo, adata_2_pseudo, gene_df, method = 'fisherz', p_value_threshold=0.05)

/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/qiuaodon/anaconda3/lib

In [38]:
#calculate the correlation between g1 and g2
calculate_gene_correlation(results, adata_1_pseudo, adata_2_pseudo)
calculate_residuals_correlation(results, adata_1_pseudo, adata_2_pseudo)

/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/scRNA_utils.py:914: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.7240284662188929' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  results.loc[index, 'g1vsg2_correlation'] = corr_g1_g2
/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/scRNA_utils.py:1060: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.2987271664939062' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  results.at[index, 'g1_residualvsg2_residuals_correlation'] = corr


,g1,g2,L,R,pValue,g1vsg2_correlation,g1_residualvsg2_residuals_correlation
0,MAFB_T,RGS1_Fibro,CD70,CD27,0.103012,0.724028,0.298727
1,CH25H_T,NEDD9_Fibro,HBEGF,ERBB4,0.257140,0.634404,0.210936
2,PMAIP1_T,NEDD9_Fibro,HBEGF,ERBB4,0.196190,0.623865,0.239512
3,BATF_T,ZC2HC1C_Fibro,TNFSF4,TNFRSF4,0.061543,0.609968,0.339313
4,CRTAM_T,LST1_Fibro,TNFSF13B,TNFRSF17,0.062461,0.597793,0.338215
...,...,...,...,...,...,...,...
1135,JUNB_T,LST1_Fibro,TNFSF13B,TNFRSF17,0.053667,-0.490336,-0.349298
1136,TSC22D3_T,UNC5CL_Fibro,CXCL12,ACKR3,0.087784,-0.491482,-0.311885
1137,TSC22D3_T,LINC01355_Fibro,THBS1,SDC1,0.059201,-0.495386,-0.342174
1138,HSPA6_T,SFRP2_Fibro,ADM,CALCRL,0.076387,-0.543124,-0.322903


In [52]:
results_filtered = results[(results['pValue'] > 0.1)]
results_filtered = results_filtered[(results_filtered['g1vsg2_correlation'] > 0.4)]
results_filtered = results_filtered[(results_filtered['g1_residualvsg2_residuals_correlation'] < 0.3)]
results_filtered = results_filtered[(results_filtered['g1_residualvsg2_residuals_correlation'] > -0.3)]
results_filtered 

,g1,g2,L,R,pValue,g1vsg2_correlation,g1_residualvsg2_residuals_correlation
0,MAFB_T,RGS1_Fibro,CD70,CD27,0.103012,0.724028,0.298727
1,CH25H_T,NEDD9_Fibro,HBEGF,ERBB4,0.257140,0.634404,0.210936
2,PMAIP1_T,NEDD9_Fibro,HBEGF,ERBB4,0.196190,0.623865,0.239512
8,CH25H_T,CTGF_Fibro,HBEGF,ERBB4,0.163898,0.562904,0.257173
9,ZFYVE16_T,HERC5_Fibro,INHBA,ACVR2B,0.328636,0.553984,0.182541
...,...,...,...,...,...,...,...
572,ITK_T,JUND_Fibro,COL1A1,ITGA3,0.140956,0.401868,0.271264
593,ARID5B_T,SGK1_Fibro,COL1A1,ITGA3,0.136083,0.400731,0.274464
597,SAT1_T,CDKN1A_Fibro,IL2,IL2RG,0.138631,0.400728,0.272781
599,SAT1_T,CDKN1A_Fibro,HBEGF,ERBB4,0.181266,0.400728,0.247403


In [53]:
results_filtered['L_R'] = results_filtered['L'] + '_' + results_filtered['R']
results_filtered['L_R'].nunique()

56

In [54]:
results_filtered.to_csv(data_dir + 'CIT_test_results_CD4EX_Fibro_results.csv')

In [57]:
import pandas as pd
import numpy as np

# Sample data (assuming results_filtered is already available)
# Adjusting the script to use absolute mean and determine final sign based on majority sign

grouped_results = results_filtered.groupby(["L", "R"])

# Create a new DataFrame to hold the GEM1 and GEM2 groupings
tracked_gem1 = set()
tracked_gem2 = set()

min_gene_num = 2

# Temporary list to hold data before creating DataFrame
temp_data = []

for (ligand, receptor), group in grouped_results:
    gem1 = group["g1"].drop_duplicates().tolist()
    gem2 = group["g2"].drop_duplicates().tolist()
    
    # Calculate mean correlation values using absolute mean
    g1vsg2_corr_mean = np.abs(group["g1vsg2_correlation"]).mean()
    g1_res_vs_g2_res_mean = np.abs(group["g1_residualvsg2_residuals_correlation"]).mean()
    p_value_mean = group["pValue"].mean()
    
    # Determine final sign based on majority sign of correlations
    sign_g1vsg2 = np.sign(group["g1vsg2_correlation"]).sum()
    sign_g1_res_vs_g2_res = np.sign(group["g1_residualvsg2_residuals_correlation"]).sum()
    
    final_g1vsg2_corr = g1vsg2_corr_mean * (1 if sign_g1vsg2 >= 0 else -1)
    final_g1_res_vs_g2_res = g1_res_vs_g2_res_mean * (1 if sign_g1_res_vs_g2_res >= 0 else -1)
    
    # Check the overlap between current GEM_g1 and previously tracked GEM_g1
    overlap_gem1 = len(set(gem1) & tracked_gem1)
    overlap_gem2 = len(set(gem2) & tracked_gem2)
    
    # Filter to include only those with at least three genes in GEM_g1
    if len(gem1) > min_gene_num and len(gem2) > min_gene_num:
        temp_data.append({
            "Ligand": ligand,
            "Receptor": receptor,
            "GEM1_elements": gem1,
            "GEM2_elements": gem2,
            "Num_genes_GEM1": len(gem1),
            "Num_genes_GEM2": len(gem2),
            "GEM1vsGEM2_corr(mean)": final_g1vsg2_corr,
            "GEM1_residualvsGEM2_residual_corr(mean)": final_g1_res_vs_g2_res,
            "average_significance": p_value_mean
        })
        # Update the tracked GEM_g1 and GEM_g2 sets
        tracked_gem1.update(gem1)
        tracked_gem2.update(gem2)

# Convert the temporary data to a DataFrame
grouped_df = pd.DataFrame(temp_data)
grouped_df

,Ligand,Receptor,GEM1_elements,GEM2_elements,Num_genes_GEM1,Num_genes_GEM2,GEM1vsGEM2_corr(mean),GEM1_residualvsGEM2_residual_corr(mean),average_significance
0,ADM,CALCRL,"[CD38_T, IL18RAP_T, NTNG2_T, PRDM1_T, SYTL3_T]","[ANGPTL4_Fibro, SPRY4_Fibro, SGK1_Fibro]",5,3,0.434287,0.189950,0.360899
1,CCL5,CCR4,"[ERN1_T, ZBTB32_T, ADGRG1_T, IL18RAP_T, LAG3_T]","[GBP2_Fibro, LST1_Fibro, IL18BP_Fibro]",5,3,0.445026,0.269824,0.145085
2,CDH1,CDH1,"[KIAA1217_T, PELI1_T, KLF10_T]","[ELF3_Fibro, PGR_Fibro, CRISPLD2_Fibro]",3,3,0.431212,0.256994,0.178962
3,COL1A1,ITGA1,"[RNF19A_T, FASLG_T, ITK_T, ZFYVE16_T]","[CRISPLD2_Fibro, RAPGEF4_Fibro, FAM157C_Fibro,...",4,4,0.423531,0.280724,0.128811
4,COL1A1,ITGA3,"[RNF19A_T, FASLG_T, ITK_T, ZFYVE16_T, TGIF1_T,...","[CRISPLD2_Fibro, RAPGEF4_Fibro, FAM157C_Fibro,...",6,5,0.415848,0.257304,0.167174
5,COL4A1,ITGA1,"[KIAA1217_T, RNF19A_T, PELI1_T, ITK_T, KLF10_T]","[ELF3_Fibro, CRISPLD2_Fibro, PGR_Fibro]",5,3,0.432148,0.271138,0.145230
6,HBEGF,ERBB4,"[CH25H_T, PMAIP1_T, TUBA3D_T, CITED2_T, FAM13A...","[NEDD9_Fibro, CTGF_Fibro, CDKN1A_Fibro, JUND_F...",6,4,0.505588,0.227727,0.236553
7,INHBA,ACVR2B,"[ZFYVE16_T, EGR1_T, SH2B3_T, FASLG_T]","[HERC5_Fibro, RAPGEF4_Fibro, FAM157C_Fibro]",4,3,0.463195,0.234329,0.216023
8,ITGB1,VCAM1,"[F2R_T, ADGRG1_T, IL18RAP_T, SFMBT2_T, LAG3_T,...","[LST1_Fibro, GBP2_Fibro, MT1A_Fibro, IL18BP_Fi...",10,4,0.434439,0.240919,0.215256
9,THBS1,SDC1,"[TSC22D3_T, FASLG_T, RGS16_T]","[TSC22D1_Fibro, RAPGEF4_Fibro, LINC01355_Fibro]",3,3,0.433840,0.294355,0.108542


In [58]:
# rank each GEM_g1 by order of name inside
grouped_df['GEM1_elements'] = grouped_df['GEM1_elements'].apply(lambda x: sorted(x))
grouped_df['GEM2_elements'] = grouped_df['GEM2_elements'].apply(lambda x: sorted(x))

In [59]:
# save the grouped_df to excel
grouped_df.to_excel(data_dir + 'CIT_results_CD4EXvsFibro_groupedLR.xlsx')